In [47]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Dense, Dropout, GaussianNoise, BatchNormalization
from tensorflow.keras import Model
import matplotlib.pyplot as plt
import seaborn as sns

In [48]:
x_train = pd.read_pickle("../metadata/x_train.pkl")
y_train = np.ravel(pd.read_pickle("../metadata/y_train.pkl"))
x_test = pd.read_pickle("../metadata/x_test.pkl")

In [49]:
inputs = Input(shape = (5,))
x = Dense(8, activation = "relu")(inputs)
x = Dense(8, activation = "relu")(x)
x = Dense(8, activation = "relu")(x)
outputs = Dense(1, activation = "sigmoid")(x)
model = Model(inputs = inputs, outputs = outputs)
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
history = model.fit(
    x = x_train,
    y = y_train,
    epochs = 20,
    verbose = 0
)

In [50]:
y_test = (model.predict(x_test, verbose = 0)>0.5).astype(int).flatten()
submission = pd.DataFrame({
    "PassengerId": range(len(x_train) + 1, len(x_train) + len(x_test) + 1, 1),
    "Survived": y_test
})
submission.to_csv("../outputs/submission.csv", index = False)

In [51]:
history.history["accuracy"][-1]

0.8181818127632141